In [ ]:
import torch 
from d2l import torch as d2l
from torch import nn

In [ ]:
def vgg_block( num_convs , input_channels , out_channels ):
    layer = []
    for _ in range( num_convs ):
        layer.append( nn.Conv2d( input_channels , out_channels , kernel_size=3 ,padding=1 ) )
        layer.append( nn.ReLU() )
        input_channels = out_channels
    layer.append( nn.MaxPool2d( kernel_size=2 , stride=2 ))
    return nn.Sequential( *layer ) #返回一个vggblock块

In [ ]:
#构建一个元组，每一个小元组对应的是两个元素，第一个元素对应一个块中卷积层的个数，第二个元素对应的是一个块的输出通道
conv_arch = ( (1 , 64) , ( 1 , 128) , (2,256) , ( 2 ,512 ) , ( 2 , 512 ) )

In [ ]:
def vggnet( conv_arch  ):
    conv_block = []
    input_channels = 1
    for ( num_convs , output_channels ) in conv_arch:
        conv_block.append( vgg_block( num_convs , input_channels , output_channels  ) )
        input_channels = output_channels
    return nn.Sequential( *conv_block , nn.Flatten(),#batchsize*512*7*7
                           nn.Linear( output_channels*7*7,  4096) , nn.ReLU() , nn.Dropout( p=0.5),
                           nn.Linear( 4096 , 4096 ),nn.ReLU( ) ,nn.Dropout( p=0.5 ),
                           nn.Linear( 4096 , 10 ) ) 

In [ ]:
net = vggnet( conv_arch )
x = torch.rand( 1,1,224,224)
for layer in net:
    x = layer( x )
    print( layer.__class__.__name__ , 'output_size:' , x.shape)

In [ ]:
#现在就尝试将通道数减小，毕竟这种通道训练起来挺慢的
redio = 4#这里注意要是2的倍数，不能奇数
small_convs_arch = [ ( item[0] , item[1]//4) for item in conv_arch]
small_convs_arch

In [ ]:
batch_size = 32
train_iter , test_iter  = d2l.load_data_fashion_mnist( batch_size=batch_size ,resize=224 )

In [9]:
lr ,  num_epochs = 0.001 , 10
d2l.train_ch6( net , train_iter , test_iter , num_epochs , lr , d2l.try_gpu(i=1) )